In [14]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [13]:
print("GPU Available:", tf.test.is_gpu_available())

GPU Available: True


Constants

In [1]:
IMAGE_FOLDER ='DataSet\Teeth Segmentation PNG\d2\img'
MASK_FOLDER = 'DataSet\Teeth Segmentation PNG\d2\masks_human'

IMAGE_DIMENSION = (512,512)

In [2]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split


def load_images_and_masks(image_folder, mask_folder, image_dim=IMAGE_DIMENSION):
    images = []
    masks = []
    
    # Get list of image files
    image_files = os.listdir(image_folder)
    
    for img_name in image_files:
        # Construct full paths
        img_path = os.path.join(image_folder, img_name)
        # Assuming mask has same name and .png extension
        mask_name = os.path.splitext(img_name)[0] + '.png' 
        mask_path = os.path.join(mask_folder, mask_name)
        
        # Check if files exist
        if not os.path.exists(img_path):
            print(f"Warning: Image file not found: {img_path}")
            continue
            
        if not os.path.exists(mask_path):
            print(f"Warning: Mask file not found: {mask_path}")
            continue
        
        # Read image
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            print(f"Warning: Could not load image: {img_path}")
            continue
            
        # Read mask
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        if mask is None:
            print(f"Warning: Could not load mask: {mask_path}")
            continue
        
        # Resize both image and mask
        img = cv2.resize(img, image_dim)
        mask = cv2.resize(mask, image_dim)
        
        # Convert mask to binary (Separates teeth (1) from background (0)) 
        mask = np.where(mask > 128, 1, 0)
        
        images.append(img)
        masks.append(mask)
    
    
    # Convert lists to numpy arrays and normalize images for better training
    images = np.array(images) / 255.0 
    masks = np.array(masks)
    
    # Since it is grayscale images, extra dimension for channels is required
    images = np.expand_dims(images, axis=-1)
    masks = np.expand_dims(masks, axis=-1)
    
    print(f"Successfully loaded {len(images)} images and masks")
    return images, masks

# Load images and masks
images, masks = load_images_and_masks(IMAGE_FOLDER, MASK_FOLDER)

# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(images, masks, test_size=0.2, random_state=42)

print(f"Training samples: {X_train.shape[0]}, Validation samples: {X_val.shape[0]}")

Successfully loaded 598 images and masks
Training samples: 478, Validation samples: 120


In [17]:
def unet_model(input_size=(512, 512, 1)):
    # Define input layer correctly
    inputs = tf.keras.Input(shape=input_size)
    
    # Rest of the model architecture remains the same
    c1 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

    c2 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)

    c3 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)

    # Bottleneck
    c4 = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c4)

    # Expansive Path (Decoder)
    u5 = tf.keras.layers.UpSampling2D((2, 2))(c4)
    u5 = tf.keras.layers.Conv2D(256, (2, 2), activation='relu', padding='same')(u5)
    u5 = tf.keras.layers.Concatenate()([u5, c3])
    c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same')(u5)
    c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c5)

    u6 = tf.keras.layers.UpSampling2D((2, 2))(c5)
    u6 = tf.keras.layers.Conv2D(128, (2, 2), activation='relu', padding='same')(u6)
    u6 = tf.keras.layers.Concatenate()([u6, c2])
    c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u6)
    c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c6)

    u7 = tf.keras.layers.UpSampling2D((2, 2))(c6)
    u7 = tf.keras.layers.Conv2D(64, (2, 2), activation='relu', padding='same')(u7)
    u7 = tf.keras.layers.Concatenate()([u7, c1])
    c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u7)
    c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c7)

    outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(c7)

    # Create model with correct input specification
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

# Compile the model
# Adam optimizer: Adaptive learning rate optimization
# Binary cross-entropy loss: Suitable for binary segmentation
# Accuracy metric: Measures correct pixel classifications
model = unet_model()
model.compile(optimizer='adam', 
             loss='binary_crossentropy',
             metrics=['accuracy'])

# Display model architecture summary
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 512, 512, 64  640         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 512, 512, 64  36928       ['conv2d[0][0]']                 
                                )                                                             

In [18]:
# Configure GPU memory growth
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

Physical devices cannot be modified after being initialized


In [19]:
tf.keras.backend.clear_session()

In [20]:
# Import necessary TensorFlow components
from tensorflow.keras.optimizers import Adam  # For model optimization
from tensorflow.keras.losses import BinaryCrossentropy  # For binary segmentation loss
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping  # For training monitoring
import tensorflow as tf
from tensorflow.keras.mixed_precision import set_global_policy
set_global_policy('mixed_float16')
# Clear any existing TensorFlow sessions to prevent memory issues
tf.keras.backend.clear_session()

# Configure the model's training parameters
model.compile(
    optimizer=Adam(learning_rate=1e-4),  # Use Adam optimizer with small learning rate
    loss=BinaryCrossentropy(),  # Binary cross-entropy loss for segmentation
    metrics=['accuracy']  # Track accuracy during training
)

# Data Augmentation Function (Increase training data variety and improve model performance)
def augment(image, mask):
    """
    Apply random transformations to both image and mask to increase training data variety
    """
    with tf.device('/CPU:0'):
        # Random 90-degree rotation (50% probability)
        if tf.random.uniform([]) > 0.5:
            image = tf.image.rot90(image)
            mask = tf.image.rot90(mask)

        # Random cropping to original size
        # This helps model learn to handle different parts of the image
        image = tf.image.random_crop(image, size=[IMAGE_DIMENSION[0], IMAGE_DIMENSION[1], 1])
        mask = tf.image.random_crop(mask, size=[IMAGE_DIMENSION[0], IMAGE_DIMENSION[1], 1])

        # Random horizontal flipping
        # Helps model learn invariance to left/right orientation
        image = tf.image.random_flip_left_right(image)
        mask = tf.image.random_flip_left_right(mask)

    return image, mask

# Dataset Creation Function
def create_dataset(X, y, batch_size=8):
    """
    Create a TensorFlow dataset with data augmentation and optimization
    """
    # Convert numpy arrays to TensorFlow dataset
    dataset = tf.data.Dataset.from_tensor_slices((X, y))
    
    # Apply augmentation to each image-mask pair
    # num_parallel_calls optimizes performance by parallel processing
    dataset = dataset.map(augment, num_parallel_calls=tf.data.AUTOTUNE)
    
    # Optimize dataset performance
    dataset = dataset.shuffle(buffer_size=len(X))  # Shuffle data
    dataset = dataset.batch(batch_size)  # Create batches
    dataset = dataset.prefetch(tf.data.AUTOTUNE)  # Prefetch next batch
    
    return dataset

# Create training and validation datasets
train_dataset = create_dataset(X_train, y_train, batch_size=4)
val_dataset = create_dataset(X_val, y_val, batch_size=4)

# Training Callbacks Setup
# ModelCheckpoint: Save the best model during training
checkpoint = ModelCheckpoint(
    'model.keras',  # Save model to this file
    save_best_only=True,  # Only save the best model
    monitor='val_loss',  # Monitor validation loss
    mode='min'  # Lower is better
)

# EarlyStopping: Stop training if model stops improving
early_stopping = EarlyStopping(
    monitor='val_loss',  # Monitor validation loss
    patience=10,  # Stop after 10 epochs without improvement
    mode='min'  # Lower is better
)

# Train the model
history = model.fit(
    train_dataset,  # Training data
    validation_data=val_dataset,  # Validation data
    epochs=50,  # Number of training epochs
    batch_size=4,  # Number of samples per batch
    callbacks=[checkpoint, early_stopping],  # Training callbacks,
    max_queue_size=10,
    workers=1
)

# Evaluate the model on validation set
loss, accuracy = model.evaluate(val_dataset)
print(f"Validation Loss: {loss}, Validation Accuracy: {accuracy}")

Epoch 1/50
120/120 [==============================] - 82s 480ms/step - loss: nan - accuracy: 0.7902 - val_loss: nan - val_accuracy: 0.7810
Epoch 2/50
120/120 [==============================] - 47s 391ms/step - loss: nan - accuracy: 0.7901 - val_loss: nan - val_accuracy: 0.7806
Epoch 3/50
120/120 [==============================] - 47s 391ms/step - loss: nan - accuracy: 0.7901 - val_loss: nan - val_accuracy: 0.7810
Epoch 4/50
120/120 [==============================] - 47s 391ms/step - loss: nan - accuracy: 0.7901 - val_loss: nan - val_accuracy: 0.7810
Epoch 5/50
120/120 [==============================] - 47s 391ms/step - loss: nan - accuracy: 0.7901 - val_loss: nan - val_accuracy: 0.7811
Epoch 6/50
120/120 [==============================] - 47s 391ms/step - loss: nan - accuracy: 0.7902 - val_loss: nan - val_accuracy: 0.7808
Epoch 7/50
120/120 [==============================] - 47s 391ms/step - loss: nan - accuracy: 0.7903 - val_loss: nan - val_accuracy: 0.7810
Epoch 8/50
120/120 [=======

In [21]:
model.save('teeth_segmentation_model_2.h5')  # HDF5 format

In [23]:
# Load the best model
best_model = tf.keras.models.load_model('teeth_segmentation_model.h5')

# Continue training
history_continued = best_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=40,  # Remaining epochs
    batch_size=2,
    max_queue_size=5,
    workers=1,
    use_multiprocessing=False,
    callbacks=[checkpoint, early_stopping]
)

Epoch 1/40
120/120 [==============================] - 48s 392ms/step - loss: nan - accuracy: 0.7901 - val_loss: nan - val_accuracy: 0.7810
Epoch 2/40
120/120 [==============================] - 47s 392ms/step - loss: nan - accuracy: 0.7904 - val_loss: nan - val_accuracy: 0.7810
Epoch 3/40
120/120 [==============================] - 47s 393ms/step - loss: nan - accuracy: 0.7901 - val_loss: nan - val_accuracy: 0.7809
Epoch 4/40
 69/120 [================>.............] - ETA: 18s - loss: nan - accuracy: 0.7872

KeyboardInterrupt: 

In [ ]:
best_model.save('teeth_segmentation_model_3.h5')

In [22]:
def plot_training_history_detailed(history):
    """
    Plot training history with additional analysis and annotations
    """
    # Set style
    plt.style.use('seaborn')
    
    # Create figure with two subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Plot Loss
    train_loss = history.history['loss']
    val_loss = history.history['val_loss']
    
    # Find best epoch for validation loss
    best_val_loss_epoch = np.argmin(val_loss) + 1
    best_val_loss = min(val_loss)
    
    ax1.plot(train_loss, label='Training Loss', color='blue', linewidth=2)
    ax1.plot(val_loss, label='Validation Loss', color='red', linewidth=2)
    ax1.set_title('Model Loss Over Time', fontsize=14, pad=15)
    ax1.set_xlabel('Epoch', fontsize=12)
    ax1.set_ylabel('Loss', fontsize=12)
    ax1.legend(fontsize=10)
    ax1.grid(True)
    
    # Add annotation for best validation loss
    ax1.annotate(f'Best Val Loss: {best_val_loss:.4f}\nEpoch: {best_val_loss_epoch}',
                xy=(best_val_loss_epoch-1, best_val_loss),
                xytext=(10, 10), textcoords='offset points',
                bbox=dict(boxstyle='round,pad=0.5', fc='yellow', alpha=0.5),
                arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0'))
    
    # Plot Accuracy
    train_acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    
    # Find best epoch for validation accuracy
    best_val_acc_epoch = np.argmax(val_acc) + 1
    best_val_acc = max(val_acc)
    
    ax2.plot(train_acc, label='Training Accuracy', color='blue', linewidth=2)
    ax2.plot(val_acc, label='Validation Accuracy', color='red', linewidth=2)
    ax2.set_title('Model Accuracy Over Time', fontsize=14, pad=15)
    ax2.set_xlabel('Epoch', fontsize=12)
    ax2.set_ylabel('Accuracy', fontsize=12)
    ax2.legend(fontsize=10)
    ax2.grid(True)
    
    # Add annotation for best validation accuracy
    ax2.annotate(f'Best Val Acc: {best_val_acc:.4f}\nEpoch: {best_val_acc_epoch}',
                xy=(best_val_acc_epoch-1, best_val_acc),
                xytext=(10, 10), textcoords='offset points',
                bbox=dict(boxstyle='round,pad=0.5', fc='yellow', alpha=0.5),
                arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0'))
    
    # Adjust layout
    plt.tight_layout()
    
    # Show the plot
    plt.show()
    
    # Print detailed metrics
    print("\nTraining Analysis:")
    print(f"Best Validation Loss: {best_val_loss:.4f} at Epoch {best_val_loss_epoch}")
    print(f"Best Validation Accuracy: {best_val_acc:.4f} at Epoch {best_val_acc_epoch}")
    print("\nFinal Metrics:")
    print(f"Training Loss: {train_loss[-1]:.4f}")
    print(f"Training Accuracy: {train_acc[-1]:.4f}")
    print(f"Validation Loss: {val_loss[-1]:.4f}")
    print(f"Validation Accuracy: {val_acc[-1]:.4f}")
    
    # Check for overfitting
    if train_loss[-1] < val_loss[-1]:
        print("\nWarning: Possible overfitting detected (training loss < validation loss)")
    if train_acc[-1] > val_acc[-1]:
        print("Warning: Possible overfitting detected (training accuracy > validation accuracy)")

# Call the detailed plotting function
plot_training_history_detailed(history)

NameError: name 'plt' is not defined

In [5]:
import tensorflow as tf

def dice_coefficient(y_true, y_pred, smooth=1e-6):
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + smooth)

def dice_coefficient_loss(y_true, y_pred):
    return 1 - dice_coefficient(y_true, y_pred)
